# Back to the Basic (Part 1. Linux)

* 2023.05.01.(Mon)
* Dept. of Math., Inha Univ.
* Byung Chun Kim (wizardbc@gmail.com)

## Could

* [VULTR](https://www.vultr.com)

### Firewall Setting
* Products > Network > Firewall
  * SSH Only

### Add your SSH key

* [VOLTR Docs - How Do I Generate SSH Keys?](https://www.vultr.com/docs/how-do-i-generate-ssh-keys/)

![ssh_keys](./resorces/ssh_keys.png)
```bash
ssh-keygen -t ed25519 -C "your_email@example.com"
cd ~/.ssh
cat id_ed25519.pub
```

### Deploy New Server

* Select Options
  * Cloud GPU
  * NVIDIA A16
  * Tokyo
  * Ubuntu 22.04 LTS x64
  * 1/16 GPU
  * `OFF` Enable Auto Backups
  * `Disable` IPv6
  * Select added SSH key
  * Firewall Group (SSH Only)
  * Enter server hostname
* Check the summary: $21.50/month
* Click `Deploy Now`

### Products

![products](./resorces/products.png)

* Select my server
* Copy `IP Address`
* Open terminal
```bash
ssh root@{ip address}
```

### Add user

```bash
adduser {username}
usermod -a -G sudo {username}
mkdir ~{username}/.ssh
chmod 700 ~{username}/.ssh
cp /root/.ssh/authorized_keys ~{username}/.ssh
chown -R {username}.{username} ~{username}/.ssh
exit

ssh {username}@{ip address}
```

### Driver
* [VULTR Docs](https://www.vultr.com/docs/introduction-to-vultr-cloud-gpus-powered-by-nvidia-a16/)

***Only for Training***
```bash
nvidia-smi

sudo apt update
sudo apt upgrade
sudo reboot

ssh {username}@{ip address}
nvidia-smi
```

* Manual Installation
```bash
mkdir -p ~/Downloads/
wget http://169.254.169.254/latest/nvidia_linux_driver_url -O ~/Downloads/nvidia_manual_driver.run
sudo bash ~/Downloads/nvidia_manual_driver.run --ui=none --no-questions
sudo reboot
ssh {username}@{ip address}
nvidia-smi
```

## Environment

### Conda

* [Miniconda](https://docs.conda.io/en/latest/miniconda.html)

```bash
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/Downloads/miniconda.sh
bash ~/Downloads/miniconda.sh
conda update -n base -c defaults conda
```

### VSCode

* [VS Code](https://code.visualstudio.com) > Extensions > Remote - SSH
  * Open Terminal ``ctrl+shift+` ``

```bash
mkdir work
```

* Open Folder

### Create Environment

#### Install [PyTorch](https://pytorch.org/get-started/locally/)

Currently PyTorch 2.0 support CUDA 11.7 and **11.8** (NOT 12.0).
```bash
conda create -n minimal python=3.10 ipykernel ipywidgets 
conda create -n minimal_torch --clone minimal
conda install -n minimal_torch pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia
conda activate minimal_torch

python
```

```python
>>> import torch
>>> torch.cuda.is_available()
True
>>> exit() # or ctrl+d
```

#### Install [JAX](https://github.com/google/jax#installation)

JAX currently ships three CUDA wheel variants:

* CUDA 12.0 and CuDNN 8.8.
* **CUDA 11.8** and CuDNN 8.6.
* CUDA 11.4 and CuDNN 8.2. This wheel is deprecated and will be discontinued with jax 0.4.8.

```bash
conda create -n torch_jax --clone minimal_torch
conda activate torch_jax
pip install --upgrade "jax[cuda11_pip]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
```

### Jupyter Notebook

* Create new file `test.ipynb`
* Select Kernel > Python Environments... > torch_jax


In [ ]:
%env XLA_PYTHON_CLIENT_PREALLOCATE=false

import numpy as np
import jax.numpy as jnp
from jax import random, device_put

size = 1000
x = np.random.normal(size=(size, size)).astype(np.float32)

In [ ]:
%timeit np.dot(x, x.T)  # runs on the CPU

In [ ]:
x = device_put(x)

In [ ]:
%timeit jnp.dot(x, x.T).block_until_ready()  # runs on the GPU